# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Make a reference to the weather_data.csv file path
csv_path = "weather_data.csv"

# Import the weather_data.csv file as a dataframe
weather_df = pd.read_csv(csv_path)

# Drop NaN values
weather_data = weather_df.dropna()

# Delete Unnamed:0 column
del weather_data["Unnamed: 0"]

# Preview dataframe
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bluff,-46.6000,168.3333,47.59,75,11,20.45,NZ,1660426786
1,Hobart,-42.8794,147.3294,52.27,97,100,21.85,AU,1660426818
2,Rio Grande,-32.0350,-52.0986,66.31,86,12,6.49,BR,1660426878
3,Kem',64.9555,34.5793,53.13,68,50,6.26,RU,1660426881
4,Kapaa,22.0752,-159.3190,82.47,69,40,12.66,US,1660426645


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [4]:
# Store Lat and Lng into locations
locations = weather_data[["Lat", "Lng"]].astype(float)

# Store Humidity values into humidity
humidity = weather_data["Humidity"].astype(float)

In [5]:
# Create a humidity heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Filter out undesirable cities
cloud_filter = weather_data.loc[weather_data["Cloudiness"] == 0]
low_temp_filter = cloud_filter.loc[cloud_filter["Max Temp"] > 60]
high_temp_filter = low_temp_filter.loc[low_temp_filter["Max Temp"] < 78]
low_wind_filter = high_temp_filter.loc[high_temp_filter["Wind Speed"] > 5]
high_wind_filter = low_wind_filter.loc[low_wind_filter["Wind Speed"] < 20]
humidity_filter = high_wind_filter.loc[high_wind_filter["Humidity"] < 60]

# Set index
vacation = humidity_filter.reset_index()
del vacation["index"]

# View dataframe
vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Akdepe,42.0551,59.3788,77.09,30,0,9.91,TM,1660426896
1,Jamestown,42.0970,-79.2353,75.02,51,0,11.50,US,1660426947
2,Lenger,42.1830,69.8839,76.44,29,0,6.91,KZ,1660427114
3,Santiago,-33.4569,-70.6483,63.84,44,0,9.22,CL,1660426822
4,Chu,43.5983,73.7614,73.78,35,0,7.96,KZ,1660427291
5,Sabzawār,36.2126,57.6819,76.28,31,0,13.80,IR,1660427362
6,Mashhad,36.2970,59.6062,69.94,43,0,6.91,IR,1660427418
7,Salta,-24.7859,-65.4117,75.79,39,0,13.80,AR,1660427423
8,Tourlaville,49.6383,-1.5664,74.62,46,0,10.36,FR,1660427614
9,Derzhavīnsk,51.1000,66.3167,67.19,49,0,5.37,KZ,1660427324


### Hotel Map
* Store into variable named `vacation`.
* Add a "Nearest Hotel" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create empty list to store hotel names
hotels = []

# Loop through each vacation city to find nearest hotel
for city in range(len(vacation["City"])):

    latitude = vacation.loc[city]["Lat"]
    longitude = vacation.loc[city]["Lng"]
    city_coordinates = f"{latitude}, {longitude}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params = {
        "location": city_coordinates,
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    
    request = requests.get(base_url, params = params)
    response = request.json()
    
    try:
        hotels.append(response["results"][0]["name"])
        
    except:
        hotels.append("Nearest hotel not found.")
        

In [10]:
# Add hotels column to vacation dataframe
vacation["Nearest Hotel"] = hotels

# View dataframe
vacation

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,Akdepe,42.0551,59.3788,77.09,30,0,9.91,TM,1660426896,Nearest hotel not found.
1,Jamestown,42.0970,-79.2353,75.02,51,0,11.50,US,1660426947,Hampton Inn & Suites Jamestown
2,Lenger,42.1830,69.8839,76.44,29,0,6.91,KZ,1660427114,Gostevoy Dom 1
3,Santiago,-33.4569,-70.6483,63.84,44,0,9.22,CL,1660426822,Hotel Panamericano
4,Chu,43.5983,73.7614,73.78,35,0,7.96,KZ,1660427291,Shyngyzkhan
5,Sabzawār,36.2126,57.6819,76.28,31,0,13.80,IR,1660427362,پزشکی قانونی سبزوار
6,Mashhad,36.2970,59.6062,69.94,43,0,6.91,IR,1660427418,Khanye Sabz Hotel
7,Salta,-24.7859,-65.4117,75.79,39,0,13.80,AR,1660427423,Hotel Alejandro I
8,Tourlaville,49.6383,-1.5664,74.62,46,0,10.36,FR,1660427614,Logis Hôtel la Régence
9,Derzhavīnsk,51.1000,66.3167,67.19,49,0,5.37,KZ,1660427324,Ooo.khim Baza


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation.iterrows()]
locations = vacation[["Lat", "Lng"]]

In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))